In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2

Using TensorFlow backend.


In [2]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# download = drive.CreateFile({'id': '1ucLenvikYCZcFLdckyMalID-_tSNZA9q'})
# download = drive.CreateFile({'id': '1_xdLPGDKXG4bYz8tuMzyeTE-ZVKlxDFi'})
download = drive.CreateFile({'id': '1F5J4WSakU32rHLIMGVMsC_f2fk8_hVXL'})
download.GetContentFile('new_train_data.tar')
# download.GetContentFile('sample.tar')

In [0]:
df_train = pd.read_csv('trainLabels.csv')

In [7]:
df_train.head()

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [8]:
df_train.shape

(35126, 2)

In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35126 entries, 0 to 35125
Data columns (total 2 columns):
image    35126 non-null object
level    35126 non-null int64
dtypes: int64(1), object(1)
memory usage: 549.0+ KB


In [0]:
# delRows = df_train[ (df_train['level'] == 1) & (df_train['level'] == 2) ].index
# df_train.drop(delRows, inplace=True)
df_train.drop( df_train[ df_train['level'] == 1 ].index , inplace=True)
df_train.drop( df_train[ df_train['level'] == 2 ].index , inplace=True)

In [0]:
df_train.loc[df_train['level'] ==3, 'level'] = 1
df_train.loc[df_train['level'] ==4, 'level'] = 1

In [12]:
df_train.shape

(27391, 2)

In [0]:
targets_series = pd.Series(df_train['level'])

In [0]:
target_labels = np.asarray(targets_series)

In [0]:
im_size1 = 256
im_size2 = 256

In [0]:
x_train = []
y_train = []
x_test = []

In [0]:
import tarfile

my_tar = tarfile.open('new_train_data.tar')
my_tar.extractall('./new_train_data') # specify which folder to extract to
my_tar.close()

In [18]:
i = 0 
for f, breed in tqdm(df_train.values):
    # print(type(cv2.imread('train/train/{}.jpeg'.format(f))))
    if type(cv2.imread('new_train_data/new_train_data/{}.jpeg'.format(f)))==type(None):
        # print("Inside if")
        continue
    else:
      img = cv2.imread('new_train_data/new_train_data/{}.jpeg'.format(f))
      # img = np.array(img)
      label = target_labels[i]
      # print("Inside else")
      # print(img)
      # x_train.append(img)
      x_train.append(cv2.resize(img, (im_size1, im_size2)))
      y_train.append(label)
      # print(label)
      i += 1
np.save('x_train2',x_train)
np.save('y_train2',y_train)
print('Done')

100%|██████████| 27391/27391 [05:09<00:00, 88.53it/s]


Done


In [0]:
x_train = np.load('x_train2.npy', allow_pickle=True)
y_train = np.load('y_train2.npy', allow_pickle=True)

In [20]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
print(x_train.dtype,type(x_train))
print(y_train.dtype)

uint8 <class 'numpy.ndarray'>
int64


In [0]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.0
# x_train_raw = x_train / 255.0

In [0]:
from keras.utils import np_utils

y_train_raw = np_utils.to_categorical(np.array(y_train_raw), 2)
# y_test = np_utils.to_categorical(np.array(y_test),2)

In [24]:
print(x_train_raw.shape)
print(y_train_raw.shape)

(1748, 256, 256, 3)
(1748, 2)


In [0]:
# x_train_raw

In [0]:
# y_train_raw

In [0]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.2, random_state=1)

In [28]:
num_class = y_train_raw.shape[1]
print(num_class)

2


In [29]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
import tensorflow as tf
tf.reset_default_graph()


# Building 'VGG Network'
network = input_data(shape=[None, im_size1, im_size2, 3])

network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2, strides=2)

network = conv_2d(network, 128, 3, activation='relu')
network = conv_2d(network, 128, 3, activation='relu')
network = max_pool_2d(network, 2, strides=2)

network = conv_2d(network, 256, 3, activation='relu')
network = conv_2d(network, 256, 3, activation='relu')
network = conv_2d(network, 256, 3, activation='relu')
network = max_pool_2d(network, 2, strides=2)

network = conv_2d(network, 512, 3, activation='relu')
network = conv_2d(network, 512, 3, activation='relu')
network = conv_2d(network, 512, 3, activation='relu')
network = max_pool_2d(network, 2, strides=2)

network = conv_2d(network, 512, 3, activation='relu')
network = conv_2d(network, 512, 3, activation='relu')
network = conv_2d(network, 512, 3, activation='relu')
network = max_pool_2d(network, 2, strides=2)

network = fully_connected(network, 1024, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 1024, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 2, activation='softmax')

network = regression(network, optimizer='rmsprop',
                     loss='categorical_crossentropy',
                     learning_rate=0.0001)









Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
keep_dims is deprecated, use keepdims instead





In [30]:
import tflearn

model = tflearn.DNN(network, tensorboard_verbose=0)
# model.summary()



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor







In [31]:
#training the model
model.fit(x_train_raw, y_train_raw, n_epoch=5, batch_size=32, shuffle=True,
          show_metric=True)

Training Step: 274  | total loss: 0.22024 | time: 59.139s
| RMSProp | epoch: 005 | loss: 0.22024 - acc: 0.9461 -- iter: 1728/1748
Training Step: 275  | total loss: 0.21378 | time: 60.230s
| RMSProp | epoch: 005 | loss: 0.21378 - acc: 0.9483 -- iter: 1748/1748
--


In [32]:
# print(Y_valid)
#evaluating the testing score
score = model.evaluate(X_valid, Y_valid)
print('Test accuarcy: ', score)

Test accuarcy:  [0.9514285734721593]


In [0]:
ypred=model.predict(X_valid)

In [0]:
y_pred2=[]
for i in ypred:
    y_pred2.append(i.argmax())
y_pred2=np.asarray(y_pred2)

In [40]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_valid, y_pred2)

array([[168,   0],
       [  7,   0]])